In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from tclab import TCLab
from package_PID import PIDController

# --- Définition des paramètres ---
Kp = 1.2  # Gain proportionnel
Ki = 0.5  # Gain intégral
Kd = 0.1  # Gain dérivé
FF_gain = 0.8  # Gain du feedforward
Ts = 1.0  # Temps d'échantillonnage

# --- Création du contrôleur PID ---
pid = PIDController(Kp, Ki, Kd, Ts, FF_gain)

# --- Initialisation du TCLab ---
lab = TCLab()
time.sleep(3)  # Laisser le système démarrer

# --- Données de simulation ---
SP = 50  # Consigne
temps = []
MVm = []
PVm = []
DV = []

print("Expérience en cours...")

try:
    for t in range(100):  # 100 itérations = 100 secondes
        start = time.time()

        # Mesurer la température actuelle (PV)
        PV = lab.T1

        # Appliquer une perturbation (DV) en temps réel
        DV_val = 10 if 50 <= t < 70 else 0
        DV.append(DV_val)

        # Calculer la commande avec le PID + Feedforward
        MV = pid.compute(SP, PV, DV_val)

        # Appliquer la commande sur TCLab
        lab.Q1(MV)
        
        # Sauvegarde des valeurs
        temps.append(t)
        MVm.append(MV)
        PVm.append(PV)

        # Attendre la fin du pas d'échantillonnage
        elapsed = time.time() - start
        time.sleep(max(0, Ts - elapsed))

finally:
    lab.Q1(0)
    lab.close()

# --- Sauvegarde des données expérimentales ---
data = np.column_stack((temps, MVm, PVm, DV))
np.savetxt("Data/TCLab_PID_FF.csv", data, delimiter=",", header="Time,MV,PV,DV", comments='')

# --- Tracé des résultats ---
plt.figure(figsize=(10, 5))
plt.plot(temps, PVm, 'g-', label="PV (Température)")
plt.plot(temps, [SP] * len(temps), 'k--', label="Consigne")
plt.plot(temps, MVm, 'b-', label="Commande (MV)")
plt.plot(temps, DV, 'r-', label="Perturbation (DV)")
plt.xlabel("Temps [s]")
plt.ylabel("Température [°C]")
plt.title("Expérience PID avec Feedforward sur TCLab")
plt.legend()
plt.grid()
plt.show()

